# Lab 13 Task 1 - Dimensionality Reduction Using AutoEncoder

### SAHIL AMRITKAR
### 20BRS1180  


In [ ]:
import pandas as pd
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

In [10]:
data=pd.read_csv('housing.csv')
data.drop(['ocean_proximity'], axis=1, inplace=True)
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0


In [11]:
TARGET_NAME = 'median_house_value'

train_data, test_data = train_test_split(data, test_size=0.2)

x_train, y_train = train_data.drop(TARGET_NAME, axis=1), train_data[TARGET_NAME]
x_test, y_test = test_data.drop(TARGET_NAME, axis=1), test_data[TARGET_NAME]

In [12]:
from sklearn.preprocessing import MinMaxScaler

def scale_datasets(x_train, x_test):
  """
  Standard Scale test and train data
  """
  standard_scaler = MinMaxScaler()
  x_train_scaled = pd.DataFrame(
      standard_scaler.fit_transform(x_train),
      columns=x_train.columns
  )
  x_test_scaled = pd.DataFrame(
      standard_scaler.transform(x_test),
      columns = x_test.columns
  )
  return x_train_scaled, x_test_scaled
  
x_train, x_test = scale_datasets(x_train, x_test)

In [16]:
x_train.shape

(16512, 8)

In [30]:
class AutoEncoders(Model):

  def __init__(self, output_units):

    super().__init__()
    self.encoder = Sequential(
        [
          Dense(32, activation="relu"),
          Dense(16, activation="relu"),
          Dense(7, activation="relu")
        ]
    )

    self.decoder = Sequential(
        [
          Dense(16, activation="relu"),
          Dense(32, activation="relu"),
          Dense(output_units, activation="sigmoid")
        ]
    )

  def call(self, inputs):

    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded

In [34]:
auto_encoder = AutoEncoders(len(x_train.columns))

auto_encoder.compile(
    loss='mae',
    metrics=['mae'],
    optimizer='adam'
)

In [35]:
history = auto_encoder.fit(
    x_train, 
    x_train, 
    epochs=15, 
    batch_size=32, 
    validation_data=(x_test, x_test)
)

Epoch 1/15
516/516 [==============================] - 1s 1ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/15
516/516 [==============================] - 0s 794us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/15
516/516 [==============================] - 0s 750us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/15
516/516 [==============================] - 0s 757us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/15
516/516 [==============================] - 0s 755us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/15
516/516 [==============================] - 0s 782us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/15
516/516 [==============================] - 0s 754us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/15
516/516 [==============================] - 0s 750us/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/15
516/516 [======

In [48]:
encoder_layer = auto_encoder.get_layer('sequential_28')
reduced_df = pd.DataFrame(encoder_layer.predict(x_train))
reduced_df = reduced_df.add_prefix('feature_')

516/516 [==============================] - 0s 373us/step


In [ ]:
reduced_df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6
0,0.821988,0.789000,1.142840,1.368817,0.125202,0.0,0.766812
1,0.704511,0.701202,1.137836,1.257343,0.000000,0.0,0.590769
2,0.884187,1.069612,1.605846,1.784462,0.893819,0.0,1.235279
3,0.883630,1.080074,1.330255,1.429193,0.715033,0.0,1.245618
4,0.775504,1.058932,1.554463,1.719027,0.678277,0.0,1.095626
...,...,...,...,...,...,...,...
16995,0.465979,0.347602,3.870452,1.631976,0.452138,0.0,1.829111
16996,0.783827,0.266121,3.370812,1.271786,0.461949,0.0,1.917097
16997,1.216018,0.137232,2.962279,0.795130,0.625288,0.0,2.249912
16998,1.204461,0.112699,3.043098,0.979922,0.621360,0.0,2.146817
